In [4]:
# import libraries
import pandas as pd
import numpy as np

In [9]:
# read data
data=pd.read_csv('divorce/divorce.csv',delimiter=";")
# get rank of matrix
print("Rank=",np.linalg.matrix_rank(data))

Rank= 55


In [11]:
# PCA
U,S,VT=np.linalg.svd(data)
# columns of U represent prinicaipal components of data